In [1]:
from os import environ
import dotenv

dotenv.load_dotenv("../.env")
GROQ_API=environ.get('GROQ_API')

In [1]:
from sqlalchemy import create_engine, MetaData

engine = create_engine('sqlite:///ecommerce.db')

In [2]:
metadata_obj = MetaData()
metadata_obj.reflect(engine)
metadata_obj.tables.keys()

dict_keys(['Clientes', 'Fornecedores', 'Funcionarios'])

In [3]:
import pandas as pd

pd.read_sql_table('Clientes', engine)

,ID_Cliente,Nome,Email,Rua_Numero,Estado,Valor_gasto
0,1,lucas moura,lucas.moura@email.com,"sitio de cardoso, 49",sao paulo,29017
1,2,julia andrade,julia.andrade@email.com,"trecho mariane teixeira, 90",tocantins,7834
2,3,rafael dias,rafael.dias@email.com,"setor de duarte, 114",paraiba,4071
3,4,carla souza,carla.souza@email.com,"recanto ana livia lopes, 53",para,17512
4,5,felipe neto,felipe.neto@email.com,"jardim de monteiro, 11",sao paulo,24307
...,...,...,...,...,...,...
95,96,ana luiza correia,heitor40@correia.net,"passarela pedro das neves, 806",para,9550
96,97,augusto nascimento,luiz-fernando91@da.br,"nucleo teixeira, 1",mato grosso do sul,22149
97,98,dr. caue pires,azevedomelissa@bol.com.br,ladeira peixoto,parana,33236
98,99,livia novaes,caldeiramelissa@costa.com,"travessa lima, 82",ceara,42653


In [4]:
pd.read_sql_table('Fornecedores', engine)

,ID_Fornecedores,Nome,Contato,Telefone,Email,Rua,Bairro,Cidade,Estado,cep
0,1,technex solutions,lucas martins,11912345678,info@technexsolutions.com,"rua das inovacoes, 123",centro,sao paulo,sp,1000000
1,2,quantum devices,isabela oliveira,11923456789,contact@quantumdevices.net,"avenida tecnologica, 456",jardim futuro,guarulhos,sp,20000000
2,3,innovatetech corporation,rafael pereira,11934567890,support@innovatetechcorp.com,"alameda do progresso, 789",centro das inovacoes,santo andre,sp,30000000
3,4,electrogadget innovations,ana silva,11945678901,sales@electrogadgetinnovations.com,"rua eletronica, 234",vila progresso,sao caetano,sp,40000000
4,5,nexustech systems,pedro almeida,11956789012,info@nexustechsystems.com,"travessa da conexao, 567",parque das maravilhas,sao paulo,sp,88000000
5,6,quantumware technologies,camila santos,11967890123,inquiries@quantumwaretech.com,"rua quantica, 890",bairro techville,guarulhos,sp,90000000


In [5]:
pd.read_sql_table('Funcionarios', engine)

,ID_Funcionarios,Nome,Cargo,DataContratacao,Telefone,Email,Rua,Bairro,Cidade,Estado,cep
0,1,carla souza,vendas,2022-03-15,1134567890,carla.souza@email.com,"rua das flores, 123",vila esperanca,sao paulo,sp,1000001
1,2,paulo silva,vendas,2022-05-10,1123456789,paulo.silva@email.com,"avenida do sol, 456",jardim da harmonia,sao paulo,sp,20000001
2,3,marta rocha,gerencia,2022-07-20,1145678901,marta.rochaa@email.com,"praca da liberdade, 789",parque dos sabias,sao paulo,sp,1000001
3,4,sofia ramos,tecnico,2022-01-12,1198765432,sofia.ramos@email.com,"alameda dos pinheiros, 234",floresta dos pinheiros,sao paulo,sp,40000001
4,5,joao pereira,estoque,2022-09-05,1123456789,joao.pereira@email.com,"travessa dos girassois, 567",centro,sao paulo,sp,20000001
5,6,talita borges,financeiro,2022-04-02,1156789012,talita.borges@email.com,"rodovia do mar, 890",nova aurora,sao paulo,sp,90000001
6,7,larissa freitas,vendas,2023-01-10,1187654321,larissa.freitas@email.com,"beco das estrelas, 1234",centro,sao paulo,sp,20000001


In [2]:
from llama_index.core import Settings
from llama_index.llms.groq import Groq
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

modelo="llama-3.3-70b-versatile"
modelo_hf_emb="BAAI/bge-m3"

Settings.llm = Groq(model=modelo, api_key = GROQ_API)
Settings.embed_model = HuggingFaceEmbedding(model_name = modelo_hf_emb)

In [7]:
from llama_index.core import SQLDatabase
from llama_index.core.objects import SQLTableNodeMapping

In [8]:
sql_database = SQLDatabase(engine)
table_node_mapping = SQLTableNodeMapping(sql_database)

In [9]:
from llama_index.core.objects import SQLTableSchema

In [10]:
table_schema_objs = []

for nome_tabelas in metadata_obj.tables.keys():
    table_schema_objs.append(SQLTableSchema(table_name=nome_tabelas))

In [11]:
from llama_index.core.objects import ObjectIndex
from llama_index.core import VectorStoreIndex

In [12]:
obj_index = ObjectIndex.from_objects(table_schema_objs, table_node_mapping, VectorStoreIndex)

In [13]:
obj_retriever = obj_index.as_retriever(similarity_top_k=1)

In [14]:
from llama_index.core.indices.struct_store.sql_query import SQLTableRetrieverQueryEngine

In [15]:
query_engine = SQLTableRetrieverQueryEngine(sql_database, obj_retriever)

In [16]:
resposta = query_engine.query('Quais estados mais frequentes na tabela Clientes?')

In [17]:
resposta.metadata

{'e641c24c-8a14-4829-a00f-a447de209a3a': {'sql_query': 'SELECT Estado, COUNT(ID_Cliente) AS Frequencia FROM Clientes GROUP BY Estado ORDER BY Frequencia DESC',
  'result': [('parana', 7),
   ('para', 7),
   ('rondonia', 6),
   ('tocantins', 5),
   ('sergipe', 5),
   ('rio de janeiro', 5),
   ('maranhao', 5),
   ('goias', 5),
   ('ceara', 5),
   ('amapa', 5),
   ('roraima', 4),
   ('rio grande do norte', 4),
   ('piaui', 4),
   ('pernambuco', 4),
   ('distrito federal', 4),
   ('sao paulo', 3),
   ('minas gerais', 3),
   ('mato grosso do sul', 3),
   ('bahia', 3),
   ('santa catarina', 2),
   ('rio grande do sul', 2),
   ('paraiba', 2),
   ('mato grosso', 2),
   ('amazonas', 2),
   ('espirito santo', 1),
   ('alagoas', 1),
   ('acre', 1)],
  'col_keys': ['Estado', 'Frequencia']},
 'sql_query': 'SELECT Estado, COUNT(ID_Cliente) AS Frequencia FROM Clientes GROUP BY Estado ORDER BY Frequencia DESC',
 'result': [('parana', 7),
  ('para', 7),
  ('rondonia', 6),
  ('tocantins', 5),
  ('sergip

In [18]:
resposta = query_engine.query('Quais as fornecedoras localizadas na cidade de São Paulo?')
print(resposta)

Não há fornecedoras localizadas na cidade de São Paulo. A lista de resultados está vazia.


FAZER A LLM GERAR AS DESCRIÇÕES DAS TABELAS E COLUNAS

In [19]:
llm = Groq(model=modelo, api_key = key)

def gerar_descricao_tabela(nome_tabela, df_amostra):
    prompt = f"""
    Analise a amostra da tabela '{nome_tabela}' abaixo e forneça uma curta e breve descrição do conteúdo dessa tabela.
    Informe até o máximo de 5 valores únicos de cada coluna.


    Amostra da Tabela:
    {df_amostra}

    Descrição:
    """

    resposta = llm.complete(prompt = prompt)

    return resposta.text

In [20]:
nomes_tabelas = metadata_obj.tables.keys()

In [21]:
dicionario_tabelas = {}

In [22]:
for nome_tabela in nomes_tabelas:
  df = pd.read_sql_table(nome_tabela,engine)
  df_amostra = df.head(5).to_string()

  descricao = gerar_descricao_tabela(nome_tabela, df_amostra)
  dicionario_tabelas[nome_tabela] = descricao
  print(f'Tabela: {nome_tabela}\n Descrição: {descricao}')
  print('-'*15)

Tabela: Clientes
 Descrição: A tabela 'Clientes' contém informações sobre clientes, incluindo identificação, contato, endereço e valor gasto. Abaixo, seguem até 5 valores únicos de cada coluna:

- **ID_Cliente**: 1, 2, 3, 4, 5
- **Nome**: lucas moura, julia andrade, rafael dias, carla souza, felipe neto
- **Email**: lucas.moura@email.com, julia.andrade@email.com, rafael.dias@email.com, carla.souza@email.com, felipe.neto@email.com
- **Rua_Numero**: sitio de cardoso, 49, trecho mariane teixeira, 90, setor de duarte, 114, recanto ana livia lopes, 53, jardim de monteiro, 11
- **Estado**: sao paulo, tocantins, paraiba, para
- **Valor_gasto**: 29017, 7834, 4071, 17512, 24307

Essa tabela parece ser usada para gerenciar informações de clientes, incluindo seus dados de contato, localização e o valor total gasto por cada cliente.
---------------
Tabela: Fornecedores
 Descrição: A tabela 'Fornecedores' contém informações sobre diferentes fornecedores, incluindo detalhes de contato e endereço. Ab

In [23]:
# Pergunta do usuário
pergunta = 'Como posso usar um filtro para mostrar apenas os clientes com idade acima de 30 anos em SQL?'

# Obtendo a resposta do modelo
resposta = llm.complete(pergunta)

# Exibindo a resposta
print(resposta)

Para mostrar apenas os clientes com idade acima de 30 anos em SQL, você pode usar a cláusula `WHERE` com uma condição que filtre as idades. Aqui está um exemplo básico de como fazer isso:

```sql
SELECT *
FROM clientes
WHERE idade > 30;
```

Nesse comando:

- `SELECT *` escolhe todas as colunas da tabela.
- `FROM clientes` especifica a tabela que você deseja consultar, que nesse caso é `clientes`.
- `WHERE idade > 30` é a condição que filtra os resultados para incluir apenas as linhas onde a idade é maior que 30.

Certifique-se de que a tabela `clientes` tenha uma coluna chamada `idade` (ou o nome que você usou para a coluna de idade) e que os dados de idade estejam armazenados de forma numérica (como `INT` ou `INTEGER`) para que a comparação funcione corretamente.

### Exemplo Prático

Suponha que você tenha a seguinte tabela `clientes`:

| id | nome      | idade |
|----|-----------|-------|
| 1  | João      | 25    |
| 2  | Maria     | 32    |
| 3  | Carlos    | 41    |
| 4  | Ana   

In [ ]:
table_schema_objs = [
    SQLTableSchema(table_name= nome_tabela,context_str= dicionario_tabelas[nome_tabela]) for nome_tabela in nomes_tabelas
]

obj_index = ObjectIndex.from_objects(table_schema_objs,table_node_mapping, VectorStoreIndex)
obj_retriever = obj_index.as_retriever(similarity_top_k=1)

query_engine_2 = SQLTableRetrieverQueryEngine(sql_database,obj_retriever)

resposta = query_engine_2.query('Quais as fornecedoras localizadas na cidade de São Paulo?')
print(resposta)

resposta = query_engine_2.query('Quantos funcionários com o cargo de vendedores temos cadastrados?')
print(resposta)

In [ ]:
texto2sql = """Dada uma pergunta em linguagem natural, crie uma consulta {dialect} sintaticamente correta para executar e, em seguida, verifique os resultados da consulta e retorne a resposta. Você pode ordenar os resultados por uma coluna relevante para retornar os exemplos mais informativos no banco de dados.

Nunca consulte todas as colunas de uma tabela específica. Pergunte apenas por algumas colunas relevantes, de acordo com a pergunta.

Preste atenção para usar apenas os nomes de colunas que você pode ver na descrição do esquema. Tenha cuidado para não consultar colunas que não existem. Preste atenção em qual coluna está em qual tabela. Além disso, qualifique os nomes das colunas com o nome da tabela quando necessário.

Use o seguinte formato, cada um em uma linha:

Pergunta: Pergunta aqui
ConsultaSQL: Consulta SQL para executar
ResultadoSQL: Resultado da ConsultaSQL
Resposta: Resposta final aqui

Use apenas as tabelas listadas abaixo.

{schema}

Pergunta: {pergunta_user}
ConsultaSQL:
"""

In [ ]:
from llama_index.core import PromptTemplate

prompt_1 = PromptTemplate(texto2sql, dialect = engine.dialect.name)
print(prompt_1.template)

Dada uma pergunta em linguagem natural, crie uma consulta {dialect} sintaticamente correta para executar e, em seguida, verifique os resultados da consulta e retorne a resposta. Você pode ordenar os resultados por uma coluna relevante para retornar os exemplos mais informativos no banco de dados.

Nunca consulte todas as colunas de uma tabela específica. Pergunte apenas por algumas colunas relevantes, de acordo com a pergunta.

Preste atenção para usar apenas os nomes de colunas que você pode ver na descrição do esquema. Tenha cuidado para não consultar colunas que não existem. Preste atenção em qual coluna está em qual tabela. Além disso, qualifique os nomes das colunas com o nome da tabela quando necessário.

Use o seguinte formato, cada um em uma linha:

Pergunta: Pergunta aqui
ConsultaSQL: Consulta SQL para executar
ResultadoSQL: Resultado da ConsultaSQL
Resposta: Resposta final aqui

Use apenas as tabelas listadas abaixo.

{schema}

Pergunta: {pergunta_user}
ConsultaSQL:

